In [76]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import csv
from sklearn.preprocessing import Imputer
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

# node_dir  = 'Pubmed-Diabetes/data/nodes_selected.csv'
# all_node_dir  = 'Pubmed-Diabetes/data/nodes.csv'
# edge_dir  = 'Pubmed-Diabetes/data/edges.csv'
# graph_dir = 'Pubmed-Diabetes/data/graph.csv'

In [109]:
contents = pd.read_csv('../cora/content.csv',delimiter=',')
cites    = pd.read_csv('../cora/cites.csv',delimiter=',')

In [123]:
def get_id(el):
    return np.where(contents['id'] == el)

## Adjanecy matrix

In [90]:
import networkx as nx
G=nx.Graph()
G.add_edges_from(cites[['source_new','target_new']].values)
G2 = G.to_undirected()

In [91]:
Gc = max(nx.connected_component_subgraphs(G2), key=len)

In [93]:
A = nx.adjacency_matrix(Gc,nodelist=contents.index)
A = np.array(A.todense())
non_zeros = np.where(np.all(A == 0, axis=1) == False)[0]

In [95]:
A = A[~np.all(A == 0, axis=1),:]
A = A[:,~np.all(A == 0, axis=0)]

In [96]:
A.shape

(2485, 2485)

In [137]:
np.savetxt('../cora/graph.csv',A,delimiter=',')

In [85]:
max(non_zeros)

2707

## Feature Selection

In [97]:
contents = contents.iloc[non_zeros]

In [100]:
contents

,id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1427,feature1428,feature1429,feature1430,feature1431,feature1432,feature1433,label,is_nn,num_labels
0,31336,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,Neural_Networks,True,0
1,1061127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Rule_Learning,False,1
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Reinforcement_Learning,False,2
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Reinforcement_Learning,False,2
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Probabilistic_Methods,False,3
5,1126012,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,Probabilistic_Methods,False,3
6,1107140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Theory,False,4
7,1102850,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Neural_Networks,True,0
8,31349,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Neural_Networks,True,0
9,1106418,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Theory,False,4


In [64]:
contents['is_ml'] = contents['label'].isin(['Neural_Networks','Reinforcement_Learning','Genetic_Algorithms'])

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
# y = contents['num_labels']
y = contents['is_ml']
X = contents.drop(['id','is_nn','is_rl','is_ga','is_theory','is_ml','label','num_labels'],axis=1)

In [67]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=15)
pca.fit(X)
X_pca = pca.transform(X)

In [68]:
X_pca.shape

(2485, 15)

In [23]:
pca_contents = pd.DataFrame(data=np.hstack((y.reshape(2485,1),X_pca)),
                   columns = ["label"] + ['feature_'+str(i) for i in range(X_pca.shape[1])])

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [26]:
pca_contents.to_csv('../cora/pca_contents.csv')

In [54]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV

print(X.shape)

lrcv = LogisticRegressionCV(penalty="l1",solver='liblinear').fit(X, y)
model = SelectFromModel(lrcv, prefit=True)
X_new = model.transform(X)
X_new.shape

(2485, 1433)


(2485, 129)

In [73]:
from sklearn.linear_model import LogisticRegression
lrcv_new = LogisticRegression(C=0.05,penalty="l1",solver='liblinear').fit(X, y)
# lrcv_new = LogisticRegression(C=0.02,penalty="l1",solver='liblinear').fit(X, y)
model_new = SelectFromModel(lrcv_new, prefit=True)
X_new_new = model_new.transform(X)
X_new_new.shape

(2485, 22)

In [74]:
selected_contents = pd.DataFrame(data=np.hstack((y.reshape(2485,1),X_new_new)),
                   columns = ["label"] + ['feature_'+str(i) for i in range(X_new_new.shape[1])])

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [75]:
selected_contents.to_csv('../cora/selected_contents_ml.csv')

In [149]:
selected_contents.shape

(2485, 20)

# PubMed

In [9]:
nodes = np.loadtxt(node_dir,delimiter=',')
edges = pd.read_csv(edge_dir)
# graph = np.loadtxt(graph_dir,delimiter=',')

In [28]:
all_nodes = pd.read_csv(all_node_dir,delimiter=',')

In [29]:
fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=0)
all_nodes = pd.DataFrame(fill_NaN.fit_transform(all_nodes))


In [32]:
all_nodes

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,12187484.0,1.0,0.093935,0.028698,0.011760,0.019375,0.063161,0.170891,0.067702,0.017555,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
1,2344352.0,1.0,0.023618,0.014784,0.030926,0.052232,0.025327,0.008620,0.014879,0.038493,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
2,14654069.0,1.0,0.102263,0.010669,0.044636,0.022996,0.013785,0.018277,0.006221,0.010738,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
3,16443886.0,2.0,0.038715,0.014075,0.023996,0.013403,0.022571,0.006989,0.011919,0.021452,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
4,2684155.0,1.0,0.030616,0.080179,0.019288,0.057942,0.019854,0.012295,0.014292,0.020564,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
5,15032912.0,1.0,0.116897,0.005201,0.018466,0.012191,0.021041,0.039660,0.074813,0.022330,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
6,17988185.0,3.0,0.007445,0.011114,0.013230,0.017365,0.032801,0.009160,0.015904,0.007713,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
7,9834350.0,3.0,0.006157,0.031884,0.026710,0.010940,0.007875,0.020420,0.036443,0.025869,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
8,16230722.0,3.0,0.010479,0.004294,0.004273,0.020031,0.004322,0.038014,0.010244,0.031140,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428
9,3542527.0,2.0,0.027970,0.013917,0.072825,0.070745,0.074849,0.017944,0.032999,0.066351,...,0.022972,0.015343,0.030595,0.011753,0.017246,0.036326,0.030719,0.035853,0.022927,0.013428


In [57]:
y = (all_nodes[1] > 1).as_matrix().astype(int)
X = all_nodes.iloc[:,2:].as_matrix()

In [59]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV

X.shape

lrcv = LogisticRegressionCV(penalty="l1",solver='liblinear').fit(X, y)
model = SelectFromModel(lrcv, prefit=True)
X_new = model.transform(X)
X_new.shape

(19717, 122)

In [98]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

X.shape

lrcv_new = LogisticRegression(C=0.1,penalty="l1",solver='liblinear').fit(X, y)
model_new = SelectFromModel(lrcv_new, prefit=True)
X_new_new = model_new.transform(X)
X_new_new.shape

(19717, 8)

In [120]:
out = np.hstack((all_nodes[0].as_matrix().reshape(19717,1),y.reshape(19717,1),X_new_new))

In [114]:
all_nodes[0].as_matrix().shape,y.shape,X_new_new.shape

((19717,), (19717,), (19717, 8))

In [121]:
np.savetxt('nodes_selected.csv',out,delimiter=',')

In [100]:
cross_val_score(LogisticRegression(C=0.1,penalty="l1",solver='liblinear'), X, y, cv=5)

array([ 0.80958418,  0.81490872,  0.80831643,  0.82120213,  0.81278539])

In [75]:
lrcv.scores_[1].shape

(3, 10)

In [71]:
lrcv.C_

array([ 10000.])

In [61]:
print ('Max:', lrcv.scores_[1].max())

Max: 0.821031806422


In [63]:
print('Inverse of the ragularization param with the best performance:',lrcv.C_[0])

Inverse of the ragularization param with the best performance: 10000.0


In [91]:
a=lrcv.Cs_
b=lrcv.scores_[1][0]
pd.DataFrame(np.vstack((a,b)))

,0,1,2,3,4,5,6,7,8,9
0,0.000100,0.000774,0.005995,0.046416,0.359381,2.782559,21.544347,166.810054,1291.549665,10000.000000
1,0.208124,0.791876,0.791876,0.806937,0.815305,0.816218,0.816066,0.816066,0.816066,0.816066


In [45]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
    lrcv, X, y, cv=10)
scores     

KeyboardInterrupt: 

In [10]:
nodes

array([[  1.21874840e+07,   1.00000000e+00,   9.39348960e-02, ...,
          6.77024800e-02,   1.75546100e-02,   9.84015120e-02],
       [  2.34435200e+06,   1.00000000e+00,   2.36179170e-02, ...,
          1.48791710e-02,   3.84934380e-02,   1.81900310e-02],
       [  1.46540690e+07,   1.00000000e+00,   1.02263144e-01, ...,
          6.22094900e-03,   1.07375460e-02,   9.34449450e-02],
       ..., 
       [  1.79341410e+07,   1.00000000e+00,   6.79419520e-02, ...,
          2.42857410e-02,   1.42676980e-02,   1.74424950e-02],
       [  1.86735440e+07,   3.00000000e+00,   9.24009900e-03, ...,
          5.92790100e-03,   1.01097040e-02,   1.44419330e-02],
       [  1.85641750e+07,   3.00000000e+00,   1.75050441e-01, ...,
          1.49800250e-02,   3.30840700e-02,   7.81088100e-03]])

In [99]:
lp = LP()
Uy_lp = np.rint(lp.closed(features,Ly))
Uy_lp_iter = np.rint(lp.iter_(features,Ly,Uy,1000))

NameError: name 'features' is not defined